In [1]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=8bc528a0ecd2afd2810b056ac8e24c0d7e25e223ef2fdc13a84d3750ed3b95ff
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:

# Imports
import os
import time
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [3]:
PARQUET_DIR = "parquet_csv"
URL = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [4]:
#initialize findspark
findspark.init()

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
#os.path.basename(URL)

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
spark.sparkContext.addFile(URL)
filepath = f"file://{SparkFiles.get(os.path.basename(URL))}"
#print(filepath) # checking where in temp folder this is located
df = spark.read.csv(filepath, header=True)
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

test_query = "select * from home_sales limit 5"
spark.sql(test_query).show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [21]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_query = """
    SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
"""
spark.sql(avg_price_query).show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [25]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg2_price_query = """
    SELECT YEAR(date_built) as year_built, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date_built)
    ORDER BY year_built
"""
spark.sql(avg2_price_query).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [26]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg3_price_query = """
    SELECT YEAR(date_built) as year_built, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(date_built)
    ORDER BY year_built
"""
spark.sql(avg3_price_query).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [28]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

query_by_view = """
  SELECT ROUND(view, 2) as view_rating, ROUND(AVG(price),2) as price
  FROM home_sales
  GROUP BY view_rating
  HAVING AVG(price) >= 350000
  ORDER BY view_rating DESC
  """

print(spark.catalog.isCached('home_sales'))
start_time_uncached = time.time()
spark.sql(query_by_view).show(5)
print("--- %s seconds ---" % (time.time() - start_time_uncached))

False
+-----------+----------+
|view_rating|     price|
+-----------+----------+
|      100.0| 1026669.5|
|       99.0|1061201.42|
|       98.0|1053739.33|
|       97.0|1129040.15|
|       96.0|1017815.92|
+-----------+----------+
only showing top 5 rows

--- 1.1011457443237305 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales. (Cache sandwich)
print(spark.catalog.isCached('home_sales'))
spark.catalog.cacheTable("home_sales")
print(spark.catalog.isCached('home_sales'))

False
True


In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
print(spark.catalog.isCached('home_sales'))
start_time_cached = time.time()
#spark.sql(query_by_view).show()
print("--- %s seconds ---" % (time.time() - start_time_cached))

True
--- 8.392333984375e-05 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet(PARQUET_DIR, mode="overwrite")

In [15]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet(PARQUET_DIR)

parquet_df.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [16]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")



In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time_parquet = time.time()



print("--- %s seconds ---" % (time.time() - start_time_parquet))

--- 0.00011444091796875 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
print(spark.catalog.isCached('home_sales'))
spark.catalog.uncacheTable("home_sales")
print(spark.catalog.isCached('home_sales'))

True
False
